In [0]:
!pip install tensorflow==2.0.0

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import sys
import os 

root = r"/content/gdrive/My Drive/ProjetoFinal"
sys.path.append(f'{root}/Code/source')
DATA_FOLDER = f"{root}/tsv/cumsum_40"

In [0]:
import tensorflow as tf
import os
from training.train_mentions import MAX_NUM_PREDICTIONS, MAX_NUM_MENTIONS, INPUT_EMBEDDINGS_SIZE, _decode_record

name_to_features = {
    "input_embeddings":
        tf.io.FixedLenFeature(MAX_NUM_MENTIONS * INPUT_EMBEDDINGS_SIZE, tf.float32),
    "input_mask":
        tf.io.FixedLenFeature(MAX_NUM_MENTIONS, tf.int64),
    "labels":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS, tf.int64),
    "output_mask":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS, tf.int64),
    "cumsum":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS * INPUT_EMBEDDINGS_SIZE * 5, tf.float32),
}

assert MAX_NUM_MENTIONS == 40

errors = []
for root, dirs, current_files in os.walk(DATA_FOLDER):
    for file_name in current_files:
      print(f"\n{root}/{file_name}")
      ds = tf.data.TFRecordDataset(filenames=[f"{root}/{file_name}"])
      ds = ds.map(lambda record: _decode_record(record, name_to_features, 0))


      # @tf.function
      def check(ds):      
          for raw_record in ds.take(2):
              print(".", end="")
              entrada, saida = raw_record
              input_embeddings = entrada["input_embeddings"]
              output_mask = entrada["output_mask"]
              cumsum = entrada["cumsum"]
              # print(tf.convert_to_tensor(output_mask).numpy())
              # print(input_embeddings)

      try:
        check(ds)
      except Exception:
        errors.append(file_name)
        !mv '{root}/{file_name}' '{DATA_FOLDER}_problems/'
        print("E")      



/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/mz/sinorama/10/mz_sinorama_10_ectb_1067_000/0.tsv

/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bn_cnn_01_cnn_0194_000.tsv
.
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bn_pri_00_pri_0056_000.tsv
.
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bc_cnn_00_cnn_0002_003.tsv
..
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/nw_wsj_10_wsj_1055_000.tsv
..
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bc_cnn_00_cnn_0002_004.tsv
.
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/pt_nt_42_nt_4207_001.tsv
..
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bc_cnn_00_cnn_0002_005.tsv
..
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/nw_xinhua_00_chtb_0075_000.tsv
..
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bc_cnn_00_cnn_0002_006.tsv
..
/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_40/old/bc_cnn_00_cnn_0002_007.tsv
.
/content/gdrive/My Drive/ProjetoFinal/

In [0]:
from training.train_mentions import get_data
from tqdm import tqdm

train_data, eval_data = get_data(DATA_FOLDER, 0.9)

for raw_record in tqdm(train_data.take(3)):
  print(repr(raw_record))